In [ ]:
import numpy as np 


### Interface Conductivity


       P            e                E
     --o------------|----------------o--

       |-- (dx)e- --|---- (dx)e+ ----|
       |------------ dx -------------|

For a steady, no-source, one-dimensional situation in which coductivity varies in a stepwise fashion from on e control volume to the next, the interface-conductivity formulat is given as



$$

k_e = \left(\frac{1-f_e}{k_P} + \frac{f_e}{k_E} \right) ^{-1}

\\
{

  f_e = \frac{(\delta x)_{e+}}{(\delta x)_{e}}
}

$$




In [ ]:
def interface_k(k_P, k_E, dx, dxe):
    f_e = dxe/dx
    return ((1-f_e)/k_P + f_e/k_E)**-1